# Import knižníc pre dátovú analýzu a vizualizáciu

Tento kód importuje základné knižnice pre komplexnú analýzu a vizualizáciu dát:

**Základné knižnice**
* `os.path` - poskytuje funkcie pre prácu s cestami k súborom v rôznych operačných systémoch
* `pandas` - umožňuje efektívnu prácu s tabuľkovými dátami
* `numpy` - poskytuje podporu pre matematické operácie a prácu s poľami

**Vizualizačné knižnice**
* `matplotlib.pyplot` - základná knižnica pre tvorbu grafov a vizualizácií
* `seaborn` - nadstavba nad matplotlib, ktorá poskytuje pokročilejšie a krajšie vizualizácie

**Konfigurácia**
Z lokálneho konfiguračného súboru sa importujú cesty k priečinkom:
* `RAW_DATA_DIR` - cesta k surovým, nespracovaným dátam
* `CLEAN_DATA_DIR` - cesta k vyčisteným dátam

In [1]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from playstore.config import RAW_DATA_DIR,CLEAN_DATA_DIR

# Načítanie dát

Kód načíta surové dáta z Google Play Store z CSV súboru, ktorý sa nachádza v priečinku definovanom v `RAW_DATA_DIR`.

In [3]:
df = pd.read_csv(os.path.join(RAW_DATA_DIR,'googleplaystore.csv')) 

# Info o datasete

Príkaz zobrazí základný prehľad o štruktúre dát - počet a typy stĺpcov, počet riadkov, a informácie o chýbajúcich hodnotách.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


# Zobrazenie názvov stĺpcov

Príkaz `df.columns` vypíše zoznam všetkých stĺpcov v našom DataFrame, čo nám umožňuje vidieť, aké informácie o aplikáciách máme k dispozícii.

In [5]:
df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

# Úprava názvov stĺpcov

Nahradzuje medzery v názvoch stĺpcov podčiarkovníkmi a zobrazí nové názvy.

In [6]:
df.columns=df.columns.str.replace(" ","_")
df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Current_Ver',
       'Android_Ver'],
      dtype='object')

# Nahradenie chýbajúcich hodnotení

Nahrádza chýbajúce hodnoty (NA) v stĺpci "Rating" mediánom všetkých hodnotení, čím zabezpečuje úplnosť dát.

In [8]:
df["Rating"].fillna(df["Rating"].median())

0        4.1
1        3.9
2        4.7
3        4.5
4        4.3
        ... 
10836    4.5
10837    5.0
10838    4.3
10839    4.5
10840    4.5
Name: Rating, Length: 10841, dtype: float64

# Kontrola chýbajúcich hodnôt

Zobrazí súčet chýbajúcich hodnôt (NA/null) pre každý stĺpec v datasete, čo nám pomáha identifikovať, kde potrebujeme dáta vyčistiť.

In [7]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content_Rating       1
Genres               0
Last_Updated         0
Current_Ver          8
Android_Ver          3
dtype: int64

# Čistenie dát

1. Odstráni všetky riadky s chýbajúcimi hodnotami 
2. Odstráni duplicitné záznamy
3. Skontroluje, či zostali nejaké chýbajúce hodnoty v datasete

In [25]:
df=df.dropna()
df=df.drop_duplicates()
df.isnull().sum()

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content_Rating    0
Genres            0
Current_Ver       0
Android_Ver       0
Updated_Day       0
Updated_Month     0
Updated_Year      0
dtype: int64

# Kontrola dátových typov

Zobrazí dátový typ (int, float, string, atď.) pre každý stĺpec v našom datasete, čo nám pomáha pochopiť, v akom formáte sú naše dáta uložené.

In [10]:
df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content_Rating     object
Genres             object
Last_Updated       object
Current_Ver        object
Android_Ver        object
dtype: object

# Čistenie údajov o veľkosti aplikácií

Tento kód upravuje a štandardizuje údaje o veľkosti aplikácií v troch krokoch:

1. Konvertuje veľkosti z 'M' (megabajty) a 'K'/'k' (kilobajty) na exponenciálny zápis
  * 'M' sa mení na 'e6' (milióny bajtov)
  * 'K' a 'k' sa menia na 'e3' (tisíce bajtov)

2. Vypočíta mediánovú hodnotu veľkosti zo všetkých aplikácií s konkrétnou veľkosťou (vynechá "Varies with device")

3. Nahrádza text "Varies with device" vypočítanou mediánovou hodnotou, čím zabezpečí konzistentnosť dát

Týmto sa všetky veľkosti štandardizujú na rovnaký formát a odstraňujú sa neštandardné hodnoty.

In [11]:
df["Size"]=df["Size"].str.replace("M","e6").str.replace("K","e3").str.replace("k","e3")
a=df[df["Size"]!="Varies with device"]["Size"].astype("float").median()
df["Size"]=df["Size"].replace("Varies with device",a)

# Počet výskytov rôznych veľkostí aplikácií

Príkaz `value_counts()` zobrazí distribúciu veľkostí aplikácií v našom datasete, konkrétne koľko aplikácií má akú veľkosť. Tento prehľad nám pomáha pochopiť, aké sú typické veľkosti aplikácií v Google Play Store a identifikovať prípadné extrémne alebo neobvyklé hodnoty. Je to užitočné pre overenie, či naše predchádzajúce úpravy veľkostí (prevod z MB a KB na štandardný formát) prebehli správne.

In [12]:
df["Size"].value_counts()

Size
14000000.0    1468
14e6           153
13e6           152
12e6           151
15e6           149
              ... 
383e3            1
454e3            1
812e3            1
442e3            1
619e3            1
Name: count, Length: 413, dtype: int64

# Konverzia veľkosti na číselný formát

Príkaz prevádza stĺpec 'Size' na číselný formát pomocou pandas funkcie `to_numeric()`. To znamená, že všetky hodnoty veľkostí, ktoré sme predtým upravili (z MB a KB), sa teraz konvertujú na čísla, s ktorými môžeme ďalej matematicky pracovať. Táto konverzia je kľúčová pre ďalšie analýzy, ako napríklad výpočet priemernej veľkosti aplikác

In [13]:
df.Size=pd.to_numeric(df["Size"])

# Zobrazenie unikátnych hodnôt inštalácií

Príkaz `unique()` zobrazí všetky rôzne hodnoty, ktoré sa vyskytujú v stĺpci 'Installs'. V tomto prípade vidíme, že počty inštalácií sú uložené ako textové reťazce s formátom ako "1,000+", "10,000+", atď., čo naznačuje, že budeme potrebovať ďalšiu úpravu týchto dát. Pred vykonaním akejkoľvek analýzy musíme tieto textové hodnoty previesť na číselný formát odstránením znakov '+' a ',' a konverziou na číselný dátový typ. Toto nám umožní vykonávať matematické operácie a analýzy nad počtom inštalácií.

In [14]:
df["Installs"].unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '100+', '500+', '10+',
       '5+', '50+', '1+'], dtype=object)

# Čistenie údajov o počte inštalácií

Tento kód čistí a konvertuje údaje o počte inštalácií aplikácií v troch krokoch, aby sme ich mohli použiť pre ďalšie analýzy:

1. Odstránenie znaku '+' z konca čísel (napr. "1000+" → "1000") pomocou lambda funkcie
2. Odstránenie oddeľovačov tisícov ',' (napr. "1,000" → "1000")
3. Konverzia očistených textových reťazcov na celé čísla pomocou funkcie int()

Týmto procesom sa textové hodnoty ako "1,000+" transformujú na číselné hodnoty (napr. 1000), s ktorými môžeme ďalej matematicky pracovať.

In [15]:
df["Installs"]=df["Installs"].apply(lambda x: x.replace("+",""))
df["Installs"]=df["Installs"].apply(lambda x: x.replace(",",""))
df["Installs"]=df["Installs"].apply(lambda x:int(x))

In [16]:
df["Installs"]

0           10000
1          500000
2         5000000
3        50000000
4          100000
           ...   
10834         500
10836        5000
10837         100
10839        1000
10840    10000000
Name: Installs, Length: 8886, dtype: int64

# Zobrazenie unikátnych cenových hodnôt

Príkaz `unique()` zobrazí všetky rôzne cenové hodnoty v datasete. Vidíme, že ceny sú uložené v textovom formáte s dolárovou značkou '$', napríklad "$0", "$0.99", "$4.99", a obsahujú aj hodnotu

In [17]:
df["Price"].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$7.99', '$5.99', '$2.99', '$3.49',
       '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49', '$10.00',
       '$24.99', '$11.99', '$79.99', '$16.99', '$14.99', '$29.99',
       '$12.99', '$2.49', '$10.99', '$1.50', '$19.99', '$15.99', '$33.99',
       '$39.99', '$3.95', '$4.49', '$1.70', '$8.99', '$1.49', '$3.88',
       '$399.99', '$17.99', '$400.00', '$3.02', '$1.76', '$4.84', '$4.77',
       '$1.61', '$2.50', '$1.59', '$6.49', '$1.29', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$8.49', '$1.75', '$14.00', '$2.00',
       '$3.08', '$2.59', '$19.40', '$3.90', '$4.59', '$15.46', '$3.04',
       '$13.99', '$4.29', '$3.28', '$4.60', '$1.00', '$2.95', '$2.90',
       '$1.97', '$2.56', '$1.20'], dtype=object)

# Úprava cenových údajov na čísla

Kód upravuje cenové údaje v datasete v dvoch krokoch:

1. Odstránenie značky dolára ($) z cenovej hodnoty pomocou lambda funkcie, ktorá aplikuje metódu `replace()` na každú hodnotu v stĺpci 'Price'
  * Napríklad: "$0.99" sa zmení na "0.99"

2. Konverzia textových hodnôt na desatinné čísla pomocou funkcie `float()`
  * Napríklad: "0.99" sa zmení na numerickú hodnotu 0.99

Po tejto úprave môžeme s cenami pracovať matematicky - počítať priemery, mediány, analyzovať cenovú distribúciu a hľadať vzťahy medzi cenou a inými vlastnosťami aplikácií.

In [18]:
df["Price"]=df["Price"].apply(lambda x: x.replace("$",""))
df["Price"]=df["Price"].apply(lambda x:float(x))

In [19]:
df["Genres"].unique()

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Auto & Vehicles', 'Beauty',
       'Books & Reference', 'Business', 'Comics', 'Comics;Creativity',
       'Communication', 'Dating', 'Education;Education', 'Education',
       'Education;Creativity', 'Education;Music & Video',
       'Education;Action & Adventure', 'Education;Pretend Play',
       'Education;Brain Games', 'Entertainment',
       'Entertainment;Music & Video', 'Entertainment;Brain Games',
       'Entertainment;Creativity', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Lifestyle;Pretend Play',
       'Adventure;Action & Adventure', 'Arcade', 'Casual', 'Card',
       'Casual;Pretend Play', 'Action', 'Strategy', 'Puzzle', 'Sports',
       'Music', 'Word', 'Racing', 'Casual;Creativity',
       'Casual;Action & Adventure', 'Simulation', 'Adventure', 'Board',
       'Trivia', 'Role Playing', 'Simulation;Education',
 

# Úprava a zobrazenie žánrov

Kód najprv vyberie z každej aplikácie len prvý žáner (pred bodkočiarkou) a následne zobrazí zoznam všetkých unikátnych žánrov v datasete, čo nám dáva jasnejší prehľad o základných kategóriách aplikácií.

In [20]:
df["Genres"]=df["Genres"].str.split(";").str[0]
df["Genres"].unique()

array(['Art & Design', 'Auto & Vehicles', 'Beauty', 'Books & Reference',
       'Business', 'Comics', 'Communication', 'Dating', 'Education',
       'Entertainment', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Adventure', 'Arcade', 'Casual', 'Card', 'Action',
       'Strategy', 'Puzzle', 'Sports', 'Music', 'Word', 'Racing',
       'Simulation', 'Board', 'Trivia', 'Role Playing', 'Educational',
       'Music & Audio', 'Video Players & Editors', 'Medical', 'Social',
       'Shopping', 'Photography', 'Travel & Local', 'Tools',
       'Personalization', 'Productivity', 'Parenting', 'Weather',
       'News & Magazines', 'Maps & Navigation', 'Casino'], dtype=object)

In [21]:
df["Genres"].value_counts()

Genres
Tools                      733
Entertainment              538
Education                  528
Action                     364
Productivity               334
Finance                    317
Communication              308
Personalization            308
Lifestyle                  306
Photography                304
Sports                     303
Medical                    302
Business                   270
Health & Fitness           264
Casual                     255
Social                     244
Arcade                     220
News & Magazines           214
Simulation                 210
Travel & Local             205
Shopping                   202
Books & Reference          179
Video Players & Editors    163
Dating                     159
Puzzle                     142
Maps & Navigation          124
Role Playing               116
Racing                     114
Food & Drink               106
Strategy                   105
Educational                 97
Adventure                   89
W

# Úprava dátumov poslednej aktualizácie

Kód spracováva dátumy poslednej aktualizácie aplikácií tak, aby boli užitočnejšie pre analýzu. Najprv konvertuje stĺpec 'Last_Updated' na skutočný dátumový formát, potom z neho extrahuje deň, mesiac a rok do samostatných stĺpcov. Pôvodný stĺpec s dátumami sa následne odstráni a zobrazí sa ukážka prvých troch riadkov upraveného datasetu pre kontrolu. Táto úprava nám umožňuje lepšie analyzovať časové vzory v aktualizáciách aplikácií.

In [22]:
df["Last_Updated"]=pd.to_datetime(df["Last_Updated"])
df["Updated_Day"]=df["Last_Updated"].dt.day
df["Updated_Month"]=df["Last_Updated"].dt.month
df["Updated_Year"]=df["Last_Updated"].dt.year
df=df.drop("Last_Updated",axis=1)
df.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Current_Ver,Android_Ver,Updated_Day,Updated_Month,Updated_Year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,Everyone,Art & Design,1.0.0,4.0.3 and up,7,1,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,Everyone,Art & Design,2.0.0,4.0.3 and up,15,1,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,Everyone,Art & Design,1.2.4,4.0.3 and up,1,8,2018


# Uloženie vyčisteného datasetu

Tento príkaz ukladá náš vyčistený a upravený dataset do nového CSV súboru s názvom 'cleaned_googleplaystore.csv'. Parameter `index=False` zabezpečuje, že sa do súboru neuložia indexové čísla riadkov, čím udržíme dáta čisté a pripravené na ďalšie použitie. Tento krok je dôležitým zavŕšením nášho procesu čistenia dát, pretože nám umožňuje zachovať všetky naše úpravy a transformácie pre budúce analýzy bez potreby opakovania procesu čistenia.

In [23]:
df.to_csv('cleaned_googleplaystore.csv', index=False)
